In [1]:
import torch
import time
import json
import pathlib
import argparse

from transformers import (
    # pipeline,
    AutoConfig,
    AutoModelForCausalLM,
    # AutoModel,
    LlamaForCausalLM,
    T5ForConditionalGeneration,
    AutoTokenizer,
    LlamaTokenizer,
)

/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id="meta-llama/Llama-2-7b-chat-hf"
config = AutoConfig.from_pretrained(model_id)
model=AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [5]:
import intel_extension_for_pytorch as ipex

/home/taosy/repo/pytorch/vision/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
model = model.eval()
model = model.to(memory_format=torch.channels_last)
model = ipex.optimize_transformers(
        model.eval(),
        dtype=torch.bfloat16,
        inplace=True,
        deployment_mode=True,
    )

/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:462: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:469: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(
/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/transformers/models/reference/modules/attentions.py:838: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1:
/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/transformers/models/reference/modules/attentio

In [7]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x _IPEXDecoderLayerCPU(
        (self_attn): _IPEXAttentionCPU(
          (rotary_emb): LlamaRotaryEmbedding()
          (_IPEXROPE): _IPEXRopeCPU(
            (embed_positions): RotaryEmbedding()
          )
          (concat_qkv): _IPEXConcatLinearCPU(
            dtype = torch.bfloat16, tpp = True, woq = False
            (linear_0): _IPEXLinear()
            (linear_1): _IPEXLinear()
            (linear_2): _IPEXLinear()
          )
          (_IPEXScaleDotProduct): _IPEXScaleDotProductCPU()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLUActivation()
        )
        (input_layernorm): _IPEXRMSNorm()
        (post_attention_layernorm): _IPEXRMSNorm()
        (mha_linear_add): _IPEXlinearAddCPU(
          dtype = torch.bfloat16, tpp = True, woq = False
          (linear): _IPEXLinear()
        )
        (mlp_linear_add): _IPEXline

In [8]:
model_id="EleutherAI/gpt-j-6b"
config = AutoConfig.from_pretrained(model_id)
model_gptj=AutoModelForCausalLM.from_pretrained(model_id)

In [9]:
print(model_gptj)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

In [11]:
model_gptj = model_gptj.eval()
model_gptj = model_gptj.to(memory_format=torch.channels_last)
model_gptj = ipex.optimize_transformers(
        model_gptj.eval(),
        dtype=torch.bfloat16,
        inplace=True,
        deployment_mode=True,
    )

/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:462: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/frontend.py:469: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(
/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/transformers/models/gptj/modeling_gptj.py:609: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:
/home/taosy/bin/miniconda3/envs/pt-39-llm/lib/python3.9/site-packages/intel_extension_for_pytorch/transformers/models/cpu/fusions/mha_fusion.py:72: TracerWarning: torch.tensor results ar

In [12]:
print(model_gptj)


GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Identity()
    (h): ModuleList(
      (0-27): 28 x _IPEXDecoderLayerCPU(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): _IPEXAttentionCPU(
          (attn_dropout): Identity()
          (resid_dropout): Identity()
          (out_proj): _IPEXLinear()
          (_IPEXROPE): _IPEXRopeCPU(
            (embed_positions): RotaryEmbedding()
          )
          (concat_qkv): _IPEXConcatLinearCPU(
            dtype = torch.bfloat16, tpp = True, woq = False
            (linear_0): _IPEXLinear()
            (linear_1): _IPEXLinear()
            (linear_2): _IPEXLinear()
          )
          (_IPEXScaleDotProduct): _IPEXScaleDotProductCPU()
        )
        (mlp): GPTJMLP(
          (act): NewGELUActivation()
          (dropout): Identity()
        )
        (linear_add_add): _IPEXlinearAddAddCPU(
          dtype = torch.bfloat16, tpp = True, woq = False
          